In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("mid").getOrCreate()
sc = spark.sparkContext

22/05/10 19:49:42 WARN Utils: Your hostname, ubuntu resolves to a loopback address: 127.0.1.1; using 192.168.108.128 instead (on interface ens33)
22/05/10 19:49:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/10 19:49:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
def select(record):
    tokens = record.split(',')
    usr = tokens[0]
    movie = tokens[1]
    rating = tokens[2]
    if(rating >= 3):
        return (usr, movie)
    else:
        return None

In [2]:
data = [('A', 5), ('A', -3), ('B', 3), ('B', -2), ('B', 21)]
rdd = sc.parallelize(data)

In [3]:
def select(record):
    key = record[0]
    value = record[1]
    if(value >0):
        return (key, (1, value))
    else:
        return None
target_rdd = rdd.map(select).filter(lambda x: x is not None).reduceByKey(lambda x,y: ((x[0] + y[0]), (x[1] + y[1]))).map(lambda x: (x[0], (x[1][0], x[1][1] / x[1][0])))

In [4]:
target_rdd.collect()

[('A', (1, 5.0)), ('B', (2, 12.0))]

In [5]:
data = [1,1,1,0,0,0,-2,-2,-2,-2,-2]
rdd=sc.parallelize(data)

In [13]:
def zpn(partition):
    z = 0
    p = 0
    n = 0
    for value in partition:
        if(value > 0):
            p+=1
        elif(value < 0):
            n+=1
        else:
            z+=1
    return [(z, p, n)]
#assume number is int type and may be in str type after loading from file
target_rdd = rdd.map(lambda x: int(x)).mapPartitions(zpn).reduce(lambda a, b: (a[0] + b[0], a[1] + b[1], a[2] + b[2]))

In [14]:
target_rdd

(3, 3, 5)

In [21]:
data = [('A', 5), ('A', -3), ('B', 3), ('B', -2), ('B', 21)]
rdd = spark.sparkContext.parallelize(data)

target_rdd = rdd.map(lambda x: (x[0], (1, x[1], x[1]))).\
reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1])).\
map(lambda x: (x[0], (x[1][0], x[1][1], x[1][1]/x[1][0])))

In [22]:
target_rdd.collect()

[('A', (2, 2, 1.0)), ('B', (3, 22, 7.333333333333333))]